In [48]:
!pip install nltk spacy pandarallel ipywidgets rank_bm25


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [49]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 28.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: en-core-web-md
    Found existing installation: en_core_web_md 3.8.0
    Uninstalling en_core_web_md-3.8.0:
      Successfully uninstalled en_core_web_md-3.8.0

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [50]:
import numpy as np
import pandas as pd

## 1.a) Import the collection set
The collection set contains metadata of CORD-19 academic papers.

The preprocessed and filtered CORD-19 dataset is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.


In [51]:
# 1) Download the collection set from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_COLLECTION_DATA = '../subtask_4b/subtask4b_collection_data.pkl'

In [52]:
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)

In [53]:
df_collection.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7718 entries, 162 to 1056448
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   cord_uid          7718 non-null   object        
 1   source_x          7718 non-null   object        
 2   title             7718 non-null   object        
 3   doi               7677 non-null   object        
 4   pmcid             4959 non-null   object        
 5   pubmed_id         6233 non-null   object        
 6   license           7718 non-null   object        
 7   abstract          7718 non-null   object        
 8   publish_time      7715 non-null   object        
 9   authors           7674 non-null   object        
 10  journal           6668 non-null   object        
 11  mag_id            0 non-null      float64       
 12  who_covidence_id  528 non-null    object        
 13  arxiv_id          20 non-null     object        
 14  label             7718 n

In [54]:
df_collection.head()

,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,label,time,timet
162,umvrwgaw,PMC,Professional and Home-Made Face Masks Reduce E...,10.1371/journal.pone.0002618,PMC2440799,18612429,cc-by,BACKGROUND: Governments are preparing for a po...,2008-07-09,"van der Sande, Marianne; Teunis, Peter; Sabel,...",PLoS One,NaN,NaN,NaN,umvrwgaw,2008-07-09,1215561600
611,spiud6ok,PMC,The Failure of R (0),10.1155/2011/527610,PMC3157160,21860658,cc-by,"The basic reproductive ratio, R (0), is one of...",2011-08-16,"Li, Jing; Blakeley, Daniel; Smith?, Robert J.",Comput Math Methods Med,NaN,NaN,NaN,spiud6ok,2011-08-16,1313452800
918,aclzp3iy,PMC,Pulmonary sequelae in a patient recovered from...,10.4103/0970-2113.99118,PMC3424870,22919170,cc-by-nc-sa,The pandemic of swine flu (H1N1) influenza spr...,2012,"Singh, Virendra; Sharma, Bharat Bhushan; Patel...",Lung India,NaN,NaN,NaN,aclzp3iy,2012-01-01,1325376000
993,ycxyn2a2,PMC,What was the primary mode of smallpox transmis...,10.3389/fcimb.2012.00150,PMC3509329,23226686,cc-by,The mode of infection transmission has profoun...,2012-11-29,"Milton, Donald K.",Front Cell Infect Microbiol,NaN,NaN,NaN,ycxyn2a2,2012-11-29,1354147200
1053,zxe95qy9,PMC,"Lessons from the History of Quarantine, from P...",10.3201/eid1902.120312,PMC3559034,23343512,no-cc,"In the new millennium, the centuries-old strat...",2013-02-03,"Tognotti, Eugenia",Emerg Infect Dis,NaN,NaN,NaN,zxe95qy9,2013-02-03,1359849600


#**Replacing synonyms**

 Using a thesaurus to look up synonyms for words in both the tweets and the scientific articles, and set all these to the same word. Replacing all synonyms with one common word means that there is more tollerance towards too tweets not using the exact same wording as the research article.

In [55]:
# Read the thesaurus file
thesaurus_file = pd.read_csv("Thesaurus_covid19_formatted_flattend.csv", delimiter=';', encoding='cp1252')

# Build the thesaurus
thesaurus = {
    row['prefLabel_en']: row['altLabel_en'].strip('[]').split(', ')
    for _, row in thesaurus_file.iterrows()
}

In [56]:
# Reverse the thesaurus for fast lookups
reverse_thesaurus = {}
for main_term, synonyms in thesaurus.items():
    for synonym in synonyms:
        reverse_thesaurus[synonym.lower()] = main_term.lower()
    reverse_thesaurus[main_term.lower()] = main_term.lower()

In [57]:
# Normalize the text using the reverse thesaurus
def normalize_text(text, reverse_thesaurus):
    words = text.lower().split()
    normalized_words = [reverse_thesaurus.get(word, word) for word in words]
    return ' '.join(normalized_words)

In [58]:
# Apply normalization directly to the title and abstract columns
df_collection['normalized_title'] = df_collection['title'].apply(
    lambda x: normalize_text(x, reverse_thesaurus)
)

df_collection['normalized_abstract'] = df_collection['abstract'].apply(
    lambda x: normalize_text(x, reverse_thesaurus)
)

df_collection[['title', 'abstract', 'normalized_title', 'normalized_abstract']].head()

,title,abstract,normalized_title,normalized_abstract
162,Professional and Home-Made Face Masks Reduce E...,BACKGROUND: Governments are preparing for a po...,professional and home-made face masks reduce e...,background: governments are preparing for a po...
611,The Failure of R (0),"The basic reproductive ratio, R (0), is one of...",the failure of r (0),"the basic reproductive ratio, r (0), is one of..."
918,Pulmonary sequelae in a patient recovered from...,The pandemic of swine flu (H1N1) influenza spr...,pulmonary sequelae in a patient recovered from...,the pandemic of swine flu (h1n1) influenza spr...
993,What was the primary mode of smallpox transmis...,The mode of infection transmission has profoun...,what was the primary mode of smallpox transmis...,the mode of infection transmission has profoun...
1053,"Lessons from the History of Quarantine, from P...","In the new millennium, the centuries-old strat...","lessons from the history of quarantine, from p...","in the new millennium, the centuries-old strat..."


In [60]:
import re
import string
import html
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import spacy
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
nltk.download('stopwords')



stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
nlp = spacy.load("en_core_web_md")


important_terms = {
    'covid', 'covid-19', 'hiv', 'rna', 'sars', 'r0', 'h1n1', 'who', 'cdc',
    'micro', 'kg', 'm/s', 'flu', 'cancer', 'aids', 'diabetes', 'malaria',
    'tuberculosis', 'hepatitis', 'pneumonia', 'leprosy', 'arthritis', 'asthma',
    'hypertension', 'obesity', 'influenza', 'hiv', 'hepatitis-b', 'coronavirus',
    'outbreak', 'pandemic', 'endemic', 'vaccine', 'antiviral', 'antibiotic',
    'surgical', 'gene', 'genome', 'mutation', 'pathogen'
}

def preprocess_text(text):
    text = re.sub(r'http\S+|www\S+', '', text)

    text = html.unescape(text)

    text = text.lower()

    text = re.sub(r'\s+', ' ', text).strip()

    text = text.translate(str.maketrans('', '', string.punctuation))

    text = ' '.join([word for word in text.split() if word not in stop_words or word in important_terms])

    text = ' '.join([stemmer.stem(word) if word not in important_terms else word for word in text.split()])

    return text

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/daniel.levin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/Users/daniel.levin/Library/Mobile Documents/com~apple~CloudDocs/MA/Semester_2/Advanced Information Retrieval/AIR_SS25/venv/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [61]:
df_collection['cleaned_title'] = df_collection['normalized_title'].fillna('').parallel_apply(preprocess_text)
df_collection['cleaned_abstract'] = df_collection['normalized_abstract'].fillna('').parallel_apply(preprocess_text)

In [77]:
filtered_df = df_collection[['title', 'cleaned_title', 'abstract', 'cleaned_abstract']]
filtered_df.head(10)

,title,cleaned_title,abstract,cleaned_abstract
162,Professional and Home-Made Face Masks Reduce E...,profession homemad face mask reduc exposur res...,BACKGROUND: Governments are preparing for a po...,background govern prepar potenti influenza pan...
611,The Failure of R (0),failur r 0,"The basic reproductive ratio, R (0), is one of...",basic reproduct ratio r 0 one fundament concep...
918,Pulmonary sequelae in a patient recovered from...,pulmonari sequela patient recov swine flu,The pandemic of swine flu (H1N1) influenza spr...,pandemic swine flu h1n1 influenza spread invol...
993,What was the primary mode of smallpox transmis...,primari mode smallpox transmiss implic biodefens,The mode of infection transmission has profoun...,mode infect transmiss profound implic effect c...
1053,"Lessons from the History of Quarantine, from P...",lesson histori quarantin plagu influenza,"In the new millennium, the centuries-old strat...",new millennium centuriesold strategi quarantin...
1589,Anxiety and Depression: Linkages with Viral Di...,anxieti depress linkag viral diseas,Anxiety and mood disorders are common in the g...,anxieti mood disord common gener popul countri...
2069,Effects of Ultraviolet Germicidal Irradiation ...,effect ultraviolet germicid irradi uvgi n95 re...,The ability to disinfect and reuse disposable ...,abil disinfect reus dispos n95 filter facepiec...
2843,Secretome of Intestinal Bacilli: A Natural Gua...,secretom intestin bacilli natur guard patholog,Current studies of human gut microbiome usuall...,current studi human gut microbiom usual consid...
2952,Long term outcomes in survivors of epidemic In...,long term outcom survivor epidem influenza h7n...,Patients who survive influenza A (H7N9) virus ...,patient world health organ surviv influenza h7...
3044,Far-UVC light: A new tool to control the sprea...,faruvc light new tool control spread airbornem...,Airborne-mediated microbial diseases such as i...,airbornemedi microbi diseas influenza tubercul...


## 1.b) Import the query set

The query set contains tweets with implicit references to academic papers from the collection set.

The preprocessed query set is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.

In [63]:
PATH_QUERY_TRAIN_DATA = '../subtask_4b/subtask4b_query_tweets_train.tsv'
PATH_QUERY_DEV_DATA = '../subtask_4b/subtask4b_query_tweets_dev.tsv'

In [64]:
df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')

In [78]:
df_query_train['normalized_tweet_text'] = df_query_train['tweet_text'].parallel_apply(
    lambda x: normalize_text(x, reverse_thesaurus)
)

df_query_dev['normalized_tweet_text'] = df_query_dev['tweet_text'].parallel_apply(
    lambda x: normalize_text(x, reverse_thesaurus)
)
df_query_train.head()


,post_id,tweet_text,cord_uid,normalized_tweet_text,cleaned_tweet_text,final_query,bm25_topk,in_topx
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5,oral care in rehabilitation medicine: oral vul...,oral care rehabilit medicin oral vulner oral m...,oral care rehabilit medicin oral vulner oral m...,"[htlvpvz5, h7hj64q5, trmwm9qq, 65gedo6u, rwgqk...",1.00
1,1,this study isn't receiving sufficient attentio...,4kfl29ul,this study isn't receiving sufficient attentio...,studi isnt receiv suffici attent reveal blackl...,studi isnt receiv suffici attent reveal blackl...,"[apqzyln2, asdcpvhx, 33znyrn8, ljcdfmbu, 296il...",0.00
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8,"thanks, xi jinping. a reminder that this study...",thank xi jinp remind studi conclud nonpharmace...,thank xi jinp remind studi conclud nonpharmace...,"[jtwb17u8, veeavho5, 8hkxbxz9, a0q61mpi, 5yscq...",1.00
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1,taiwan - a population of 23 million has had ju...,taiwan popul 23 million 600 case 7 death wides...,taiwan popul 23 million 600 case 7 death wides...,"[lsgm7y5t, l5ogbl5p, l4y7v729, 0w9k8iy1, x14iy...",0.25
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69,obtaining a diagnosis of autism in lower incom...,obtain diagnosi autism lower incom countri tak...,obtain diagnosi autism lower incom countri tak...,"[tiqksd69, b0dzhsrh, aqbhxv1f, k7smwz6w, 0u330...",1.00


In [66]:
df_query_train['cleaned_tweet_text'] = df_query_train['tweet_text'].parallel_apply(preprocess_text)
df_query_train['cleaned_tweet_text'] = df_query_train['cleaned_tweet_text'].fillna('').astype(str)

df_query_dev['cleaned_tweet_text'] = df_query_dev['tweet_text'].parallel_apply(preprocess_text)
df_query_dev['cleaned_tweet_text'] = df_query_dev['cleaned_tweet_text'].fillna('').astype(str)


# 2)The following code runs a BM25 after preprcoessing



In [67]:
from rank_bm25 import BM25Plus

corpus = df_collection[['cleaned_title', 'cleaned_abstract']].parallel_apply(
    lambda x: f"{x['cleaned_title']} {x['cleaned_abstract']}",
    axis=1
).tolist()

cord_uids = df_collection[:]['cord_uid'].tolist()

tokenized_corpus = [doc.split(' ') for doc in corpus]

bm25 = BM25Plus(tokenized_corpus)

The relevant documents are retrived using the BM25 model.

In [68]:
text2bm25top = {}
def get_top_cord_uids(query):
  if query in text2bm25top.keys():
      return text2bm25top[query]
  else:
      tokenized_query = query.split(' ')

      #based on the tokenized query we call the BM25 to compute the relevance score for each document relevant to the query
      doc_scores = bm25.get_scores(tokenized_query)
      # it sort the doc scores in descending order
      indices = np.argsort(-doc_scores)[:50]
      #This line uses the indices from the previous step to retrieve the actual document IDs
      bm25_topk = [cord_uids[x] for x in indices]

      text2bm25top[query] = bm25_topk
      return bm25_topk

In [69]:
df_query_train['final_query'] = df_query_train['cleaned_tweet_text']
df_query_dev['final_query'] = df_query_dev['cleaned_tweet_text']

df_query_train.head()

,post_id,tweet_text,cord_uid,normalized_tweet_text,cleaned_tweet_text,final_query
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5,oral care in rehabilitation medicine: oral vul...,oral care rehabilit medicin oral vulner oral m...,oral care rehabilit medicin oral vulner oral m...
1,1,this study isn't receiving sufficient attentio...,4kfl29ul,this study isn't receiving sufficient attentio...,studi isnt receiv suffici attent reveal blackl...,studi isnt receiv suffici attent reveal blackl...
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8,"thanks, xi jinping. a reminder that this study...",thank xi jinp remind studi conclud nonpharmace...,thank xi jinp remind studi conclud nonpharmace...
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1,taiwan - a population of 23 million has had ju...,taiwan popul 23 million 600 case 7 death wides...,taiwan popul 23 million 600 case 7 death wides...
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69,obtaining a diagnosis of autism in lower incom...,obtain diagnosi autism lower incom countri tak...,obtain diagnosi autism lower incom countri tak...


In [70]:
df_query_train['bm25_topk'] = df_query_train['final_query'].parallel_apply(lambda x: get_top_cord_uids(x))
df_query_dev['bm25_topk'] = df_query_dev['final_query'].parallel_apply(lambda x: get_top_cord_uids(x))

# 3) Evaluating the baseline
The following code evaluates the BM25 retrieval baseline on the query set using the Mean Reciprocal Rank score (MRR@5).

In [71]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        d_performance[k] = data["in_topx"].mean()
    return d_performance

In [72]:
# Evaluate retrieved candidates using MRR@k
results_train = get_performance_mrr(df_query_train, 'cord_uid', 'bm25_topk')
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'bm25_topk')

# Printed MRR@k results
print(f"Results on the train set: {dict((k, float(v)) for k, v in results_train.items())}")
print(f"Results on the dev set: {dict((k, float(v)) for k, v in results_dev.items())}")


Results on the train set: {1: 0.5720843382867813, 5: 0.6242822687310355, 10: 0.6298665681163931}
Results on the dev set: {1: 0.5657142857142857, 5: 0.6176071428571428, 10: 0.623095804988662}


In [73]:
def results_to_markdown_table(train_results, dev_results):
    # Header
    table = "| Set   | Top-K | Score     |\n"
    table += "|--------|--------|------------|\n"

    # Train rows
    for k, score in train_results.items():
        table += f"| Train | {k}     | {score:.4f} |\n"

    # Dev rows
    for k, score in dev_results.items():
        table += f"| Dev   | {k}     | {score:.4f} |\n"

    return table

print(results_to_markdown_table(results_train, results_dev))

| Set   | Top-K | Score     |
|--------|--------|------------|
| Train | 1     | 0.5721 |
| Train | 5     | 0.6243 |
| Train | 10     | 0.6299 |
| Dev   | 1     | 0.5657 |
| Dev   | 5     | 0.6176 |
| Dev   | 10     | 0.6231 |



# 4) Exporting results to prepare the submission on Codalab

In [74]:
df_query_dev['preds'] = df_query_dev['bm25_topk'].apply(lambda x: x[:5])

In [75]:
df_query_dev[['post_id', 'preds']].to_csv('predictions.tsv', index=None, sep='\t')

In [76]:
df_query_dev[['post_id', 'preds']].head()

,post_id,preds
0,16,"[25aj8rj5, o4vvlmr4, 66g5lpm6, vmmwtdia, trrg1..."
1,69,"[r58aohnu, p0kg6dyz, s2vckt2w, yrowv62k, eay6q..."
2,73,"[mkwgkkoi, gruir7aw, xavegbty, vx1hjh26, myqm6..."
3,93,"[3sr2exq9, sv48gjkk, tx8ypqsm, z795y51f, k0f4c..."
4,96,"[ybwwmyqy, ouvq2wpq, rs3umc1x, sxx3yid9, vabb2..."
